In [ ]:
# Built-in
import re, sys, math, json, os
from time import gmtime, strftime

# 3rd Party
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import Image
from IPython.display import display
import urllib.request

# Amazon
import boto3
import sagemaker
from sagemaker import get_execution_role
from sagemaker.predictor import csv_serializer
from sagemaker.serializers import CSVSerializer

In [ ]:
sys.version

In [ ]:
# Configuration
prefix = 'sagemaker/DEMO-xgboost-dm'
bucket_name = 'my-s3-bucket-name1'
s3_path = f's3://{bucket_name}/{prefix}'
s3_data_train = f'{s3_path}/train'
s3_output = f'{s3_path}/output'

## Setup

In [ ]:
# Initialize Session
b3_session = boto3.Session()
my_region = session.region_name # set the region of the instance
sm_session = sagemaker.Session()
print(type(b3_session))
print(type(sm_session))

In [ ]:
# Define IAM role
role = get_execution_role()
print(repr(role))

In [ ]:
# this line automatically looks for the XGBoost image URI and builds an XGBoost container.
xgboost_container = sagemaker.image_uris.retrieve("xgboost", my_region, "latest")
print(repr(xgboost_container))

In [ ]:
print("Success - the MySageMakerInstance is in the '" + my_region + "' region.")
print("You will use the '" + xgboost_container + "' container for your SageMaker endpoint.")

## Get Data

In [ ]:
# Create S3 Bucket
s3 = boto3.resource('s3')
print(type(s3))
try:
    if  my_region == 'us-east-1':
        s3.create_bucket(Bucket=bucket_name)
    else: 
        s3.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={ 'LocationConstraint': my_region })
    bucket = s3.Bucket(bucket_name)
    print(type(bucket))
    print('S3 bucket created successfully')
except Exception as e:
    print('S3 error: ',e)

In [ ]:
# Download and import data
try:
    urllib.request.urlretrieve ("https://d1.awsstatic.com/tmt/build-train-deploy-machine-learning-model-sagemaker/bank_clean.27f01fbbdf43271788427f3682996ae29ceca05d.csv", "bank_clean.csv")
    print('Success: downloaded bank_clean.csv.')
except Exception as e:
    print('Data load error: ',e)

try:
    model_data = pd.read_csv('./bank_clean.csv',index_col=0)
    print('Success: Data loaded into dataframe.')
except Exception as e:
    print('Data load error: ',e)

In [ ]:
model_data.info()

In [ ]:
# Train-test split with shuffle
model_data_shuffled = model_data.sample(frac=1, random_state=1729)
split_idx = int(0.7 * len(model_data))
train_data, test_data = np.split(model_data_shuffled, [split_idx])
print(train_data.shape, test_data.shape)

In [ ]:
# Format training data (e.g. target values in first column)
(pd.concat(
        [
            train_data['y_yes'], 
            train_data.drop(['y_no', 'y_yes'], axis=1)
        ], axis=1)
   .to_csv('train.csv', index=False, header=False)
)

# Upload training data to S3 so it can be used with SageMaker
(s3.Bucket(bucket_name)
   .Object(os.path.join(prefix, 'train/train.csv'))
   .upload_file('train.csv')
)
print(type(s3.Bucket(bucket_name)))
print(type(s3.Bucket(bucket_name).Object(os.path.join(prefix, 'train/train.csv'))))

s3_input_train = sagemaker.inputs.TrainingInput(s3_data=s3_data_train, content_type='csv')
print(type(s3_input_train))

## Build and Fit Estimator

In [ ]:
xgb = sagemaker.estimator.Estimator(
    xgboost_container,role, 
    instance_count=1, 
    instance_type='ml.m4.xlarge',
    output_path=s3_output,
    sagemaker_session=sm_session)
print(type(xgb))

xgb.set_hyperparameters(
    max_depth=5,
    eta=0.2,
    gamma=4,
    min_child_weight=6,
    subsample=0.8,
    silent=0,
    objective='binary:logistic',
    num_round=100)

In [ ]:
%%time
xgb.fit({'train': s3_input_train})

In [ ]:
%%time
xgb_predictor = xgb.deploy(initial_instance_count=1,instance_type='ml.m4.xlarge')
print()
print(type(xgb_predictor))

In [ ]:
test_data_array = test_data.drop(['y_no', 'y_yes'], axis=1).values #load the data into an array
xgb_predictor.serializer = CSVSerializer() # set the serializer type

predictions = xgb_predictor.predict(test_data_array).decode('utf-8') # predict!
predictions_array = np.fromstring(predictions[1:], sep=',') # and turn the prediction into an array

print(predictions_array.shape)

In [ ]:
cm = pd.crosstab(
    index=test_data['y_yes'], 
    columns=np.round(predictions_array), 
    rownames=['Observed'], 
    colnames=['Predicted'])

tn = cm.iloc[0,0]; 
fn = cm.iloc[1,0]; 
tp = cm.iloc[1,1]; 
fp = cm.iloc[0,1]; 
p = (tp+tn)/(tp+tn+fp+fn)*100
print("\n{0:<20}{1:<4.1f}%\n".format("Overall Classification Rate: ", p))
print("{0:<15}{1:<15}{2:>8}".format("Predicted", "No Purchase", "Purchase"))
print("Observed")
print("{0:<15}{1:<2.0f}% ({2:<}){3:>6.0f}% ({4:<})".format("No Purchase", tn/(tn+fn)*100,tn, fp/(tp+fp)*100, fp))
print("{0:<16}{1:<1.0f}% ({2:<}){3:>7.0f}% ({4:<}) \n".format("Purchase", fn/(tn+fn)*100,fn, tp/(tp+fp)*100, tp))

In [ ]:
display(cm)

## Clean Up

In [ ]:
xgb_predictor.delete_endpoint(delete_endpoint_config=True)
bucket.objects.all().delete()